This is MLP method

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

In [2]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df['Metadata_experiment_type'].value_counts()
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']

# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]

/tmp/ipykernel_766446/275198509.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [3]:
#A549
df_train=df_proceed[df_proceed['Metadata_Plate'].isin(
['BR00116991',
 'BR00116992',
 'BR00116993',
 'BR00116994',
 'BR00117015',
 'BR00117016',
 'BR00117017',
 'BR00117019',
 'BR00117000',
 'BR00117003',
 'BR00117004',
 'BR00117005',
 'BR00118041',
 'BR00118042',
 'BR00118043',
 'BR00118044'])]
df_test=df_proceed[df_proceed['Metadata_Plate'].isin(['BR00116995',
 'BR00117010',
 'BR00117011',
 'BR00117012',
 'BR00117013',
 'BR00117024',
 'BR00117025',
 'BR00117026',
 'BR00116996',
 'BR00116997',
 'BR00116998',
 'BR00116999',
 'BR00118045',
 'BR00118046',
 'BR00118047',
 'BR00118048'])]

In [4]:
df_negcon=df_negcon.reset_index()
df_test=df_test.reset_index()
df_test=pd.concat([df_test,df_negcon])

In [5]:
# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
testX=df_test.drop('Metadata_gene', axis = 1)
y_test = df_test['Metadata_gene']
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

In [6]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)
# Train the MLPClassifier model
mlp_clf = MLPClassifier(hidden_layer_sizes=(1280, 630, 300),
                        max_iter=300, activation='relu', verbose=True, tol=1e-8,
                        solver='adam')
mlp_clf.fit(trainX_scaled, trainY)

Iteration 1, loss = 4.54962940
Iteration 2, loss = 3.57082870
Iteration 3, loss = 2.88730756
Iteration 4, loss = 2.30196493
Iteration 5, loss = 1.85204920
Iteration 6, loss = 1.44144712
Iteration 7, loss = 1.10430656
Iteration 8, loss = 0.84788559
Iteration 9, loss = 0.65170890
Iteration 10, loss = 0.45506399
Iteration 11, loss = 0.33737970
Iteration 12, loss = 0.25937239
Iteration 13, loss = 0.16171725
Iteration 14, loss = 0.12363069
Iteration 15, loss = 0.10175569
Iteration 16, loss = 0.06481050
Iteration 17, loss = 0.05727644
Iteration 18, loss = 0.03972696
Iteration 19, loss = 0.03168179
Iteration 20, loss = 0.02144119
Iteration 21, loss = 0.01848314
Iteration 22, loss = 0.01877777
Iteration 23, loss = 0.01914988
Iteration 24, loss = 0.01515322
Iteration 25, loss = 0.01131341
Iteration 26, loss = 0.01154984
Iteration 27, loss = 0.01229711
Iteration 28, loss = 0.00726746
Iteration 29, loss = 0.00809721
Iteration 30, loss = 0.00567124
Iteration 31, loss = 0.00483076
Iteration 32, los

/home/yuchen.yang@insilico.ai/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(1280, 630, 300), max_iter=300, tol=1e-08,
              verbose=True)

In [7]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test.iloc[:,df_test.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test.iloc[:,0:df_test.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test['Metadata_experiment_type']], axis=1)

In [8]:
feature_new.to_csv('MLP_A549_new.csv')
df_test.to_csv('MLP_A549_old.csv')    

This is SLPP Method

In [9]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df['Metadata_experiment_type'].value_counts()
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']

# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
#A549
df_train=df_proceed[df_proceed['Metadata_Plate'].isin(
['BR00116991',
 'BR00116992',
 'BR00116993',
 'BR00116994',
 'BR00117015',
 'BR00117016',
 'BR00117017',
 'BR00117019',
 'BR00117000',
 'BR00117003',
 'BR00117004',
 'BR00117005',
 'BR00118041',
 'BR00118042',
 'BR00118043',
 'BR00118044'])]
df_test=df_proceed[df_proceed['Metadata_Plate'].isin(['BR00116995',
 'BR00117010',
 'BR00117011',
 'BR00117012',
 'BR00117013',
 'BR00117024',
 'BR00117025',
 'BR00117026',
 'BR00116996',
 'BR00116997',
 'BR00116998',
 'BR00116999',
 'BR00118045',
 'BR00118046',
 'BR00118047',
 'BR00118048'])]
# Train_test pre-process
df_negcon=df_negcon.reset_index()
df_test=df_test.reset_index()
df_test=pd.concat([df_test,df_negcon])
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
testX=df_test.drop('Metadata_gene', axis = 1)
y_test = df_test['Metadata_gene']
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_766446/143922095.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [10]:
X_train = trainX
y_train = y_train.reset_index()
y_train=y_train['Metadata_gene'].tolist()
n_components=300
sigma=1.0

In [11]:
## code from: github.com/tanyapole/reproduce-OSLPP/blob/main/OSLPP.py
import scipy 

def get_l2_norm(features:np.ndarray): return np.sqrt(np.square(features).sum(axis=1)).reshape((-1,1))

def get_l2_normalized(features:np.ndarray): return features / get_l2_norm(features)

def get_PCA(features, dim):
    result = PCA(n_components=dim).fit_transform(features)
    assert len(features) == len(result)
    return result

def get_W(labels,):
    W = (labels.reshape(-1,1) == labels).astype(np.int32)
    negative_one_idxs = np.where(labels == -1)[0]
    W[:,negative_one_idxs] = 0
    W[negative_one_idxs,:] = 0
    return W

def get_D(W): return np.eye(len(W), dtype=np.int32) * W.sum(axis=1)

def fix_numerical_assymetry(M): return (M + M.transpose()) * 0.5

def get_projection_matrix(features, labels, proj_dim):
    N, d = features.shape
    X = features.transpose()
    
    W = get_W(labels)
    D = get_D(W)
    L = D - W

    A = fix_numerical_assymetry(np.matmul(np.matmul(X, D), X.transpose()))
    B = fix_numerical_assymetry(np.matmul(np.matmul(X, L), X.transpose()) + np.eye(d))
    assert (A.transpose() == A).all() and (B.transpose() == B).all()

    w, v = scipy.linalg.eigh(A, B)
    assert w[0] < w[-1]
    w, v = w[-proj_dim:], v[:, -proj_dim:]
    assert np.abs(np.matmul(A, v) - w * np.matmul(B, v)).max() < 1e-5

    w = np.flip(w)
    v = np.flip(v, axis=1)

    for i in range(v.shape[1]):
        if v[0,i] < 0:
            v[:,i] *= -1
    return v

def project_features(P, features):
    # P: pca_dim x proj_dim
    # features: N x pca_dim
    # result: N x proj_dim
    return np.matmul(P.transpose(), features.transpose()).transpose()

y_train_array = np.array(y_train)
P = get_projection_matrix(trainX, y_train_array, n_components)

/tmp/ipykernel_766446/1419865204.py:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  negative_one_idxs = np.where(labels == -1)[0]


In [12]:
## transformed seen_test_data
transformed_data = np.dot(testX, P)
new_feature=pd.DataFrame(transformed_data)
testX1=df_test.iloc[:,:df_test.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX1=testX1.reset_index()
dfnew =pd.concat([testX1,new_feature],axis=1)

In [13]:
dfnew.iloc[:,1:].to_csv('SLPP_A549_new.csv')          # After feature transformation
df_test.to_csv('SLPP_A549_old.csv')                  # Before feature transformation